# Ejercicios 1 - Parte 2: Representaciones Espaciales II (soluciones)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/fundamentos-robotica-python/blob/main/3-Representaciones-Espaciales/4-ejercicios-parte2-soluciones.ipynb)

Oscar E. Ramos Ponce, Universidad de Ingeniería y Tecnología - UTEC
Fundamentos de Robótica

Este archivo contiene las soluciones del conjunto de Ejercicios: Representaciones Espaciales (II)

In [1]:
import numpy as np
import sympy as sp

In [2]:
cos = np.cos
sin = np.sin
pi = np.pi

def rotx(ang):
    Rx = np.array([[1, 0, 0],
                   [0, cos(ang), -sin(ang)],
                   [0, sin(ang), cos(ang)]])
    return Rx

def roty(ang):
    Ry = np.array([[cos(ang), 0, sin(ang)],
                   [0, 1, 0],
                   [-sin(ang), 0, cos(ang)]])
    return Ry

def rotz(ang):
    Rz = np.array([[cos(ang), -sin(ang), 0],
                   [sin(ang), cos(ang), 0],
                   [0,0,1]])
    return Rz

### Pregunta 1

In [3]:
# Función para calcular el eje y ángulo dada una matriz de rotación
# Notar que no hay verificación de singularidades (habrá problemas si sen(th)=0)
def ejeang(R):
    c = (R[0,0]+R[1,1]+R[2,2]-1.0)/2.0
    s = np.sqrt((R[1,0]-R[0,1])**2+(R[2,0]-R[0,2])**2+(R[2,1]-R[1,2])**2)/2.0
    th = np.arctan2(s,c)
    u = 1.0/(2.*sin(th))*np.array([R[2,1]-R[1,2], R[0,2]-R[2,0], R[1,0]-R[0,1]])
    return th,u

# Matriz de rotación resultante
R = roty(np.deg2rad(90)).dot(rotz(np.deg2rad(45)))
print("Matriz de rotación:"); print(np.round(R,3))

Matriz de rotación:
[[ 0.    -0.     1.   ]
 [ 0.707  0.707  0.   ]
 [-0.707  0.707  0.   ]]


In [4]:
# Eje y ángulo
th, u = ejeang(R)
print("Ángulo: ", str(th))
print("Eje:"); print(u)

Ángulo:  1.7177715174584016
Eje:
[0.35740674 0.86285621 0.35740674]


A pesar de no ser necesario en este problema, se verificará el resultado realizando la operación contraria; es decir, calculando la matriz de rotación dado el eje y ángulo obtenido. Se debe "regresar" a la matriz original.

In [5]:
# Matriz antisimétrica
def skew(u):
    su = np.array([[    0, -u[2],  u[1]],
                   [ u[2],     0, -u[0]],
                   [-u[1],  u[0],    0]])
    return su

# Fórmula de Rodrigues (forma matricial)
def rodrigues(th, u):
    s = skew(u)
    R = np.eye(3) + s*sin(th) + s.dot(s)*(1-cos(th))
    return R

In [6]:
# Matriz de rotación a partir del resultado anterior
R1 = rodrigues(1.7177, np.array([0.35740674, 0.86285621, 0.35740674]))

print(np.round(R1,3))
# Se puede verificar que esta matriz R1 es igual a la matriz R anterior

[[ 0.    -0.     1.   ]
 [ 0.707  0.707 -0.   ]
 [-0.707  0.707  0.   ]]


### Pregunta 2

In [7]:
# a) Matriz de rotación equivalente
R = rotz(np.pi/2).dot(roty(np.pi/6)).dot(rotz(np.pi/4))
print(np.round(R,3))

[[-0.707 -0.707  0.   ]
 [ 0.612 -0.612  0.5  ]
 [-0.354  0.354  0.866]]


In [8]:
# b) El x del sistema rotado, con respecto al sistema base (fijo, inicial) es la primera 
#    columna de la matriz de rotación obtenida en a

print("Dirección de x: ", R[:,0])

Dirección de x:  [-0.70710678  0.61237244 -0.35355339]


### Pregunta 3

In [9]:
TBA = np.array([[1,    0,   0,  2],
                [0,  0.6, 0.8, -1],
                [0, -0.8, 0.6,  1],
                [0,    0,   0,  1]])

PB = np.array([[2, 4, 5, 1]]).T

PA = np.linalg.inv(TBA) @ PB
print("El punto en el sistema A es:\n ", PA[0:3].flatten())

El punto en el sistema A es:
  [ 0.  -0.2  6.4]


### Pregunta 4

In [10]:
def rot2quat(R):
    w = 0.5*np.sqrt(1+R[0,0]+R[1,1]+R[2,2])
    ex = 1/(4*w)*(R[2,1]-R[1,2])
    ey = 1/(4*w)*(R[0,2]-R[2,0])
    ez = 1/(4*w)*(R[1,0]-R[0,1])
    return np.array([w, ex, ey, ez])

# a)
R = np.array([[0.7905, -0.3864,  0.4752],
              [0.6046,  0.3686, -0.7061],
              [0.0977,  0.8455,  0.5250]])
Q = rot2quat(R)

print("El cuaternión equivalente es: ", np.round(Q,3))

El cuaternión equivalente es:  [0.819 0.474 0.115 0.302]


In [11]:
# b) Se verificará obteniendo la matriz de rotación a partir del cuaternión resultante

w = Q[0]; ex = Q[1]; ey = Q[2]; ez = Q[3]
R1 = np.array([[2*(w**2+ex**2)-1,   2*(ex*ey-w*ez),    2*(ex*ez+w*ey)],
               [  2*(ex*ey+w*ez), 2*(w**2+ey**2)-1,    2*(ey*ez-w*ex)],
               [  2*(ex*ez-w*ey),   2*(ey*ez+w*ex), 2*(w**2+ez**2)-1]])

# Al mostrar el resultado, se observa que es igual a la matriz R inicial
print( np.round(R1,4))

[[ 0.7905 -0.3864  0.4752]
 [ 0.6046  0.3686 -0.7061]
 [ 0.0977  0.8455  0.525 ]]


### Pregunta 5

In [12]:
# Las rotaciones de los ángulos ZXZ son:
R = rotz(0).dot( rotx(np.pi/2) ).dot( rotz(0))
# que es equivalente a una sola rotación alrededor del eje X. 

# Esta rotación tendría theta = 90° y eje = (1, 0, 0). En la definición de cuaternión:
#    Q = (cos(theta/2), ux*sin(theta/2), uy*sin(theta/2), uz*sin(theta/2))
# Se tendrá: Q = (cos(theta/2), sin(theta/2), 0,0)

theta = np.pi/2
Q = np.array([np.cos(theta/2), np.sin(theta/2), 0, 0])
print("Cuaternión equivalente: ", np.round(Q,3))

Cuaternión equivalente:  [0.707 0.707 0.    0.   ]


### Pregunta 6 

In [13]:
# a) Se debe verificar que la parte correspondiente a la matriz de rotación sea efectivamente una matriz de rotación
T1 = np.array([[         -1/2, 0, -np.sqrt(3)/2,  1],
               [            0, 1,             0, -2],
               [-np.sqrt(3)/2, 0,          -1/2, -1],
               [            0, 0,             0,  1]]) 

T2 = np.array([[         -1/2, 0, -np.sqrt(3)/2,  0],
               [            0, 1,             0,  1],
               [ np.sqrt(3)/2, 0,          -1/2,  0],
               [            0, 0,             0,  1]]) 

T3 = np.array([[         -1/2, 0, -np.sqrt(3)/2,  1],
               [            0, 1,             0,  0],
               [ np.sqrt(3)/2, 0,          -1/2, -1],
               [            0, 0,             0,  1]])

# Para T1:
R1 = T1[0:3,0:3]
print("determinante R1: ", np.linalg.det(R1) )
print("R1*R1^T: \n", np.round( R1.dot(R1.T),3) )

# Para T2:
R2 = T2[0:3,0:3]
print("\ndeterminante R2: ", np.linalg.det(R2) )
print("R2*R2^T: \n", np.round( R2.dot(R2.T),3) )

# Para T3: la parte correspondiente a la rotación es igual que para T2
# Viendo los resultados:
# * R1 no es una matriz de rotación, y por tanto T1 no es una transformación homogénea válida
# * R2 sí es una matriz de rotación, y por tanto T2 y T3 son válidas

determinante R1:  -0.49999999999999994
R1*R1^T: 
 [[1.    0.    0.866]
 [0.    1.    0.   ]
 [0.866 0.    1.   ]]

determinante R2:  0.9999999999999999
R2*R2^T: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [14]:
# b) La conversión a cuaternión unitario se puede realizar con la función de la pregunta 4
Q = rot2quat(R2)
print("El cuaternión es: ", Q)

El cuaternión es:  [ 0.5        0.        -0.8660254  0.       ]


In [15]:
# c) Usando la función de la pregunta 1:
ang, u = ejeang(R2)
print("El ángulo es: ", np.rad2deg(ang))
print("El eje es: ", u)

El ángulo es:  120.00000000000001
El eje es:  [ 0. -1.  0.]


### Pregunta 7

In [16]:
# a) Según los datos del problema, la nueva posición y orientación del sistema 2, con respecto al sistema 1 será
T12 = np.array([[0, -1, 0,   0],
                [1,  0, 0, 0.4],
                [0,  0, 1, 0.2],
                [0,  0, 0,   1]])

# La transformación homogénea del sistema 1 con respecto al sistema 3 es (por inspección del diagrama):
T31 = np.array([[0, 1,  0, -0.5],
                [1, 0,  0,  0.5],
                [0, 0, -1,    2],
                [0, 0,  0,    1]])
                
# El nuevo sistema 2 con respecto a 3 será (el resultado se puede verificar por inspección del diagrama):
T32 = np.dot(T31, T12)
print(T32)

[[ 1.   0.   0.  -0.1]
 [ 0.  -1.   0.   0.5]
 [ 0.   0.  -1.   1.8]
 [ 0.   0.   0.   1. ]]


In [17]:
# b) El sistema 1 con respecto al sistema 0 es:
T01 = np.array([[1, 0, 0, 0],
                [0, 1, 0, 1],
                [0, 0, 1, 1],
                [0, 0, 0, 1]])
                
# El nuevo sistema 2 con respecto a 0 será:
T02 = np.dot(T01, T12)
print(T02)

[[ 0.  -1.   0.   0. ]
 [ 1.   0.   0.   1.4]
 [ 0.   0.   1.   1.2]
 [ 0.   0.   0.   1. ]]


### Pregunta 8

In [18]:
# Definición de las matrices de rotación en y, z usando sympy
def sroty(ang):
    Ry = sp.Matrix([[sp.cos(ang), 0, sp.sin(ang)],
                    [0, 1, 0],
                    [-sp.sin(ang), 0, sp.cos(ang)]])
    return Ry

def srotz(ang):
    Rz = sp.Matrix([[sp.cos(ang), -sp.sin(ang), 0],
                   [sp.sin(ang), sp.cos(ang), 0],
                   [0,0,1]])
    return Rz

def srotx(ang):
    Rx = sp.Matrix([[1, 0, 0],
                    [0, sp.cos(ang), -sp.sin(ang)],
                    [0, sp.sin(ang), sp.cos(ang)]])
    return Rx

In [19]:
# Generación de variables simbólicas
p1, p2, p3 = sp.symbols("p1 p2 p3")

# a) Rotación deseada ZXZ
R = sp.simplify(srotz(p1)*srotx(p2)*srotz(p3))
R

Matrix([
[-sin(p1)*sin(p3)*cos(p2) + cos(p1)*cos(p3), -sin(p1)*cos(p2)*cos(p3) - sin(p3)*cos(p1),  sin(p1)*sin(p2)],
[ sin(p1)*cos(p3) + sin(p3)*cos(p1)*cos(p2), -sin(p1)*sin(p3) + cos(p1)*cos(p2)*cos(p3), -sin(p2)*cos(p1)],
[                           sin(p2)*sin(p3),                            sin(p2)*cos(p3),          cos(p2)]])

b) Se considerará $R=\begin{bmatrix} r_{11} & r_{12} & r_{13} \\ r_{21} & r_{22} & r_{23} \\ r_{31} & r_{32} & r_{33}\end{bmatrix}$. Comparando términos se tiene:

* $r_{31}=\sin(\phi_2)\sin(\phi_3)$, $r_{32}=\sin(\phi_2)\cos(\phi_3)$, de donde $\sin(\phi_3) = r_{31}/\sin(\phi_2)$ y $\cos(\phi_3) = r_{32}/\sin(\phi_2)$. De allí se obtiene la función tangente, que se despeja como: $\phi_3=\text{atan2}\left(\frac{r_{31}}{\sin(\phi_2)}, \frac{r_{32}}{\sin(\phi_2)}\right)$

* De manera similar, se obtiene: $\phi_1=\text{atan2}\left(\frac{r_{13}}{\sin(\phi_2)}, -\frac{r_{23}}{\sin(\phi_2)}\right)$

* Si se eleva al cuadrado $r_{31}$ y $r_{32}$, y aplicando propiedades trigonométricas, se tiene: $r_{31}^2+r_{32}^2=\sin^2(\phi_2)$, de donde $\sin(\phi_2)=\pm \sqrt{r_{31}^2+r_{32}^2}$. Dado que $r_{33}=\cos(\phi_2)$, se tendrá: $\phi_2 = \text{atan2}\left(\pm\sqrt{r_{31}^2+r_{32}^2}, r_{33}\right)$

### Pregunta 9

In [20]:
R = np.array([[ 0.7905,  0.6046, 0.0977],
              [-0.3864,  0.3686, 0.8455],
              [ 0.4752, -0.7061, 0.5250]])

# a) Cuaternión equivalente
w = 0.5*np.sqrt(1 + R[0,0] + R[1,1] + R[2,2])
ex = 1/(4*w) * (R[2,1]-R[1,2])
ey = 1/(4*w) * (R[0,2]-R[2,0])
ez = 1/(4*w) * (R[1,0]-R[0,1])

# Cuaternión:
Q = np.array([w, ex, ey, ez])
print("Cuaternión equivalente:", np.round(Q, 3))

Cuaternión equivalente: [ 0.819 -0.474 -0.115 -0.302]


In [21]:
# b) Dado que se tiene el cuaternión equivalente, se puede usar el cuaternión 
e = np.array([ex, ey, ez])
u = e / np.linalg.norm(e)    # Eje

th = 2 * np.arctan2(np.linalg.norm(e), w)    # Ángulo en radianes
th = th/np.pi*180    # En grados

print("Eje:", np.round(u, 3))
print("Ángulo:", np.round(th, 2))

Eje: [-0.826 -0.201 -0.527]
Ángulo: 70.0


In [22]:
# c)
Rinv = R.T
print("Matriz inversa:\n", Rinv)

Matriz inversa:
 [[ 0.7905 -0.3864  0.4752]
 [ 0.6046  0.3686 -0.7061]
 [ 0.0977  0.8455  0.525 ]]


In [23]:
# Cuaternión asociado con la matriz inversa:
w = 0.5*np.sqrt(1 + Rinv[0,0] + Rinv[1,1] + Rinv[2,2])
ex = 1/(4*w) * (Rinv[2,1]-Rinv[1,2])
ey = 1/(4*w) * (Rinv[0,2]-Rinv[2,0])
ez = 1/(4*w) * (Rinv[1,0]-Rinv[0,1])

# Cuaternión:
Q = np.array([w, ex, ey, ez])
print("Cuaternión equivalente:", np.round(Q, 3))

# Se puede ver que el cuaternión tiene la misma parte escalar que en la parte a), pero el negativo de la parte vectorial

Cuaternión equivalente: [0.819 0.474 0.115 0.302]


### Pregunta 10

In [24]:
# Verificación de la rotación
p1, p2, p3 = sp.symbols("p1 p2 p3")
R = sp.simplify(srotz(p1)*sroty(p2)*srotx(p3))
R

Matrix([
[cos(p1)*cos(p2), -sin(p1)*cos(p3) + sin(p2)*sin(p3)*cos(p1), sin(p1)*sin(p3) + sin(p2)*cos(p1)*cos(p3)],
[sin(p1)*cos(p2),  sin(p1)*sin(p2)*sin(p3) + cos(p1)*cos(p3), sin(p1)*sin(p2)*cos(p3) - sin(p3)*cos(p1)],
[       -sin(p2),                            sin(p3)*cos(p2),                           cos(p2)*cos(p3)]])

Se asocia los elementos $r_{ij}$ de la matriz de rotación genérica con los elementos mostrados para obtener términos seno y coseno de cada ángulo por separado. Operando, se llega a:

$\phi_2 = \text{atan2}\left(-r_{31}, \pm \sqrt{r_{33}^2+r_{32}^2}\right)$

$\phi_3 = \text{atan2}\left( \dfrac{r_{32}}{\cos\phi_2}, \dfrac{r_{33}}{\cos\phi_2} \right)$ 

$\phi_1 = \text{atan2}\left( \dfrac{r_{21}}{\cos\phi_2}, \dfrac{r_{11}}{\cos\phi_2} \right)$ 

In [25]:
# Con los ángulos dados, se puede obtener la siguiente matriz de rotación
p1 = 10/180*np.pi; p2 = 70/180*np.pi; p3 = 30/180*np.pi
R = rotz(p1).dot(roty(p2)).dot(rotx(p3))
R

array([[ 0.33682409,  0.31232456,  0.88825835],
       [ 0.05939117,  0.93445649, -0.35108939],
       [-0.93969262,  0.17101007,  0.29619813]])

In [26]:
# Primer conjunto de ángulos
phi2 = np.rad2deg( np.arctan2(-R[2,0], np.sqrt(R[2,2]**2+R[2,1]**2)) )
phi3 = np.rad2deg( np.arctan2(R[2,1]/np.cos(phi2), R[2,2]/np.cos(phi2)) )
phi1 = np.rad2deg( np.arctan2(R[1,0]/np.cos(phi2), R[0,0]/np.cos(phi2)) )
np.array([phi1, phi2, phi3])

array([10., 70., 30.])

In [27]:
# Segundo conjunto de ángulos
phi2 = np.rad2deg( np.arctan2(-R[2,0], -np.sqrt(R[2,2]**2+R[2,1]**2)) )
phi3 = np.rad2deg( np.arctan2(R[2,1]/np.cos(phi2), R[2,2]/np.cos(phi2)) )
phi1 = np.rad2deg( np.arctan2(R[1,0]/np.cos(phi2), R[0,0]/np.cos(phi2)) )
np.array([phi1, phi2, phi3])

array([-170.,  110., -150.])

### Pregunta 11

In [28]:
def Trotx(ang):
    Tx = np.array([[1, 0, 0, 0],
                   [0, cos(ang), -sin(ang), 0],
                   [0, sin(ang), cos(ang), 0],
                   [0, 0, 0, 1]])
    return Tx

def Troty(ang):
    Ty = np.array([[cos(ang), 0, sin(ang), 0],
                   [0, 1, 0, 0],
                   [-sin(ang), 0, cos(ang), 0],
                   [0, 0, 0, 1]])
    return Ty

def Trotz(ang):
    Tz = np.array([[cos(ang), -sin(ang), 0, 0],
                   [sin(ang), cos(ang), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    return Tz

def Trasl(x,y,z):
    T = np.array([[1, 0, 0, x],
                  [0, 1, 0, y], 
                  [0, 0, 1, z],
                  [0, 0, 0, 1]])
    return T

In [29]:
ang = np.arctan(3/4)

# a)
# Transformación que describe {1} con respecto a {2}: se comienza con {2} y se termina en {1}
T21 = Trasl(0,5,0) @ Trotx(np.pi/2+ang) @ Troty(np.pi) @ Trasl(-4,0,0)
np.round(T21, 3)

array([[-1. ,  0. ,  0. ,  4. ],
       [ 0. , -0.6,  0.8,  5. ],
       [ 0. ,  0.8,  0.6, -0. ],
       [ 0. ,  0. ,  0. ,  1. ]])

In [30]:
# Transformación que describe {3} con respecto a {0}: se comienza con {0} y se termina en {3}
T03 = Trasl(0,4,0) @ Troty(-np.pi/2) @ Trotx(-(np.pi-ang))
np.round(T03, 3)

array([[ 0. ,  0.6,  0.8,  0. ],
       [ 0. , -0.8,  0.6,  4. ],
       [ 1. , -0. , -0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ]])

In [31]:
# Transformación que describe {1} con respecto a {0}: se comienza con {0} y se termina en {1}
T01 = Trasl(3,0,0) @ Troty(-np.pi/2) @ Trotx(np.pi/2)
np.round(T01, 3)

array([[ 0., -1., -0.,  3.],
       [ 0.,  0., -1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [32]:
# b) Usando composición de matrices:
T23 = T21 @ np.linalg.inv(T01) @ T03
np.round(T23, 3)

# El resultado se puede verificar por inspección (x3 está en dirección de -x2, y3 en la dirección de y2, 
# z3 en la dirección de -z1)

array([[-1.,  0., -0.,  4.],
       [ 0.,  1., -0., -0.],
       [ 0., -0., -1.,  0.],
       [ 0.,  0.,  0.,  1.]])